In [ ]:
import os
import time
import base64
import requests
import io

from dotenv import load_dotenv
from PIL import Image

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



from pytesseract import pytesseract
from groq import Groq

In [ ]:
pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [ ]:
# Path to ChromeDriver
driver_path = 'C:\\Users\\karan\\Downloads\\chromedriver-win64\\chromedriver.exe'

In [ ]:
# Email Configuration
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
GMAIL_ADDRESS = "karanganesh97@gmail.com"  # Your Gmail address
GMAIL_PASSWORD = "idnt lpfi ppbv zelw"    # Gmail password or App Password
OUTLOOK_ADDRESS = "ganesh7@illinois.edu"  # Your Outlook email address

    


In [ ]:
# Connect to an already open Chrome browser

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)
print("Browser is now connected to the script. Navigate to your LinkedIn job search page manually.")

In [ ]:
def capture_full_page_screenshot(page_number):
    try:
        screenshot_path = f"screenshot_page_{page_number}.png"
        # Scroll down to the bottom of the page to load all content
        scroll_pause_time = 2
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Capture the full page screenshot
        total_height = driver.execute_script("return document.body.scrollHeight")
        driver.set_window_size(1920, total_height)
        screenshot_data = driver.get_screenshot_as_png()

        # Save the screenshot using PIL
        img = Image.open(io.BytesIO(screenshot_data))
        img.save(screenshot_path)
        print(f"Screenshot saved at: {screenshot_path}")
        return screenshot_path
    except Exception as e:
        print(f"Error capturing full page screenshot: {e}")
        return None


In [ ]:

GROQ_API_KEY = "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"
MODEL_ID = "llama-3.2-90b-vision-preview"
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

print(GROQ_API_KEY)

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not found. Please check your environment variables.")

# Initialize the Groq client
client = Groq(api_key=GROQ_API_KEY)

def parse_screenshot_with_groq(image_path, page_number):
    try:
        # Open image and convert to byte string using PIL
        with Image.open(image_path) as img:
            img_buffer = io.BytesIO()
            img.save(img_buffer, format="JPEG")
            img_bytes = img_buffer.getvalue()

        # Convert image bytes to base64 encoded string
        encoded_image = base64.b64encode(img_bytes).decode("utf-8")

         # Prepare the data for Groq API
        data = {
            "model": MODEL_ID,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Extract job details (title, company, time posted) from this image."},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
                    ]
                }
            ]
        }

        # Make the API request to Groq
        response = requests.post(
            GROQ_API_URL,
            headers={
                "Authorization": f"Bearer {GROQ_API_KEY}",
                "Content-Type": "application/json"
            },
            json=data
        )

        response.raise_for_status()
        response_data = response.json()
        print(f"Raw response from Groq on Page {page_number}: {response_data}")

        # Extract the response
        job_details = response_data.get("result")
        
        # Assuming job details contain structured data about jobs
        if job_details:
            print(f"Jobs found on Page {page_number}: {job_details}")
            return job_details
        else:
            print(f"No jobs found on Page {page_number}.")
            return None

    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 401:
            print("Error: Unauthorized. Please check your API key.")
        elif response.status_code == 404:
            print("Error: Not Found. Please check the endpoint URL.")
        else:
            print(f"HTTP error occurred: {http_err}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error processing screenshot with Groq: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None
            
       



In [ ]:
# Function to filter jobs that are posted within the last 30 minutes
def filter_recent_jobs(jobs):
    """
    Filters jobs that are posted within the last 30 minutes.
    """
    try:
        recent_jobs = [
            job for job in jobs
            if "minute" in job.get("time_posted", "").lower() and int(job["time_posted"].split()[0]) <= 30
        ]
        return recent_jobs
    except Exception as e:
        print(f"Error filtering jobs: {e}")
        return []





In [ ]:
def navigate_to_page(page_number):
    """
    Navigate to a specific page by clicking the pagination button.
    Includes handling for dynamic elements and retries.
    """
    try:
        print(f"Attempting to navigate to Page {page_number}...")
        pagination_buttons = driver.find_elements(By.CSS_SELECTOR, "ul.artdeco-pagination__pages li")

        if not pagination_buttons:
            print("No pagination buttons found. Stopping navigation.")
            return

        for button in pagination_buttons:
            if button.text.strip() == str(page_number):
                button.click()
                time.sleep(3)  # Allow time for the page to load
                print(f"Successfully navigated to Page {page_number}.")
                return
        print(f"Page {page_number} not found in the pagination.")
    except Exception as e:
        print(f"Error navigating to Page {page_number}: {e}")

def ensure_navigation_starts_from_first_page():
    """
    Navigate to Page 1 to ensure the script starts at the first page.
    """
    try:
        print("Ensuring we start from Page 1...")
        navigate_to_page(1)
    except Exception as e:
        print(f"Error ensuring navigation starts from Page 1: {e}")

def wait_for_pagination():
    """
    Wait until the pagination element is present on the page to avoid errors.
    """
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.artdeco-pagination__pages"))
        )
        print("Pagination loaded successfully.")
    except Exception as e:
        print(f"Pagination did not load: {e}")

In [ ]:
def send_email(jobs):
    """
    Sends an email with the extracted job details.
    """
    try:
        msg = MIMEMultipart()
        msg['From'] = GMAIL_ADDRESS
        msg['To'] = OUTLOOK_ADDRESS
        msg['Subject'] = f"Job Alert: {len(jobs)} New Jobs Found!"

        body = "Hi,\n\nHere are the new jobs found on LinkedIn:\n\n"
        for idx, job in enumerate(jobs, start=1):
            body += f"Job {idx}:\n"
            body += f"Title: {job['title']}\n"
            body += f"Company: {job['company']}\n"
            body += f"Posted: {job['time_posted']}\n"
            body += f"Link/Page: {job['link']}\n\n"
        body += "Best regards,\nYour Automated Job Checker"

        msg.attach(MIMEText(body, 'plain'))

        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        print(f"Email sent to {OUTLOOK_ADDRESS}.")
    except Exception as e:
        print(f"Failed to send email: {e}")


In [ ]:
# Function to test the email functionality
def test_email_functionality():
    """
    Runs a test of the email function using the first page of job listings.
    """
    try:
        print("Running email test with the first page...")
        ensure_navigation_starts_from_first_page()  # Start from Page 1

        screenshot_path = capture_full_page_screenshot(1)  # Capture screenshot for Page 1

        if not screenshot_path:
            print("Skipping Page 1 due to screenshot error.")
            return

        # Parse jobs from screenshot using Ollama
        parsed_jobs = parse_screenshot_with_groq(screenshot_path, page_number=1)

        # Filter jobs posted within the last 30 minutes
        if parsed_jobs:
            recent_jobs = filter_recent_jobs(parsed_jobs)

            if recent_jobs:
                send_email(recent_jobs)
            else:
                print("No recent jobs found during test email functionality.")
        else:
            print("No jobs found during test email functionality.")
    except Exception as e:
        print(f"Error during test email functionality: {e}")


In [ ]:
def main():
    # Wait for user to manually navigate to the page
    input("Please navigate to the desired page and press Enter to continue...")

    # Start main processing loop
    print("Starting main processing loop...")
    total_pages = 8  # Total number of pages to process

    for page in range(1, total_pages + 1):
        print(f"Processing Page {page}...")

        screenshot_path = capture_full_page_screenshot(page)

        if not screenshot_path:
            print(f"Skipping Page {page} due to screenshot error.")
            continue

        # Parse the screenshot with Groq
        parsed_jobs = parse_screenshot_with_groq(screenshot_path, page_number=page)

        if not parsed_jobs:
            print(f"No jobs found on Page {page}.")
            continue

        # Assuming recent_jobs is a list of dictionaries containing job details
        recent_jobs = [parsed_jobs]  # Update this to properly handle parsed data if needed

        if recent_jobs:
            print(f"Recent jobs on Page {page}: {recent_jobs}")
        else:
            print(f"No recent jobs found on Page {page}.")

        # Clean up the screenshot after processing
        os.remove(screenshot_path)
        print("Processing completed.")
    
    driver.quit()

if __name__ == "__main__":
    try:
        main()  # Start the main processing loop
    except NameError:
        print("main function is not defined. Skipping main processing loop.")